# How to use `autoreload`

I have been confused on how to use [`autoreload`](http://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html) IPython extension for a long time. The documentation simply wasn't clear to me. Or, rather, it seemed clear, but then I was surprised by the behavior.


After submitting a bug request w/ [Daniel Margala](https://github.com/dmargala) (we were confused together), we got a [response](https://github.com/ipython/ipython/issues/10395#issuecomment-286467407) that helped to clear things up. 

I've updated this with my new understanding. Hopefully this helps anyone else who is confused.

In [1]:
import os
import sys
import time
sys.path.append("..")
%reload_ext autoreload

Create a simple packe with a few simple modules that we will update. 

In [2]:
directory = "../examplepackage/"
if not os.path.exists(directory):
    os.makedirs(directory)

In [3]:
%%writefile ../examplepackage/neato.py

def torpedo():
    print('First module modification 0!')

Overwriting ../examplepackage/neato.py


In [4]:
%%writefile ../examplepackage/neato2.py

def torpedo2():
    print('Second module modification 0!')

Overwriting ../examplepackage/neato2.py


In [5]:
%%writefile ../examplepackage/neato3.py

def torpedo3():
    print('Third module modification 0!')

Overwriting ../examplepackage/neato3.py


In [6]:
# when hitting 'run all' this needs a short delay (probable race condition).
time.sleep(1.5)

# `%autoreload 1`

The docs say:

```
%autoreload 1

Reload all modules imported with %aimport every time before executing the Python code typed.
```

In [7]:
import examplepackage.neato
import examplepackage.neato2
import examplepackage.neato3

In [8]:
%autoreload 1
%aimport examplepackage

You might think that importing `examplepackage` would result in that package being auto-reloaded if you updated code inside of it. You'd be wrong. Follow along!

In [9]:
examplepackage.neato.torpedo()

First module modification 0!


In [10]:
examplepackage.neato2.torpedo2()

Second module modification 0!


In [11]:
examplepackage.neato3.torpedo3()

Third module modification 0!


In [12]:
%%writefile ../examplepackage/neato.py

def torpedo():
    print('First module modification 1')

Overwriting ../examplepackage/neato.py


In [13]:
%%writefile ../examplepackage/neato2.py

def torpedo2():
    print('Second module modification 1')

Overwriting ../examplepackage/neato2.py


In [14]:
%%writefile ../examplepackage/neato3.py

def torpedo3():
    print('Third module modification 1!')

Overwriting ../examplepackage/neato3.py


In [15]:
# when hitting 'run all' this needs a short delay (probable race condition).
time.sleep(1.5)

In [16]:
examplepackage.neato.torpedo()

First module modification 0!


In [17]:
examplepackage.neato2.torpedo2()

Second module modification 0!


In [18]:
examplepackage.neato3.torpedo3()

Third module modification 0!


Nothing is updated. You have to import the module explicitly like: 

In [19]:
%autoreload 1
%aimport examplepackage.neato

In [20]:
examplepackage.neato.torpedo()

First module modification 1


In [21]:
examplepackage.neato2.torpedo2()

Second module modification 0!


In [22]:
examplepackage.neato3.torpedo3()

Third module modification 0!


#  `%autoreload 2`

The docs say: 

```
%autoreload 2

Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
```

I read this as "if you set `%autoreload 2`, then it will reload all modules except whatever you `%aimport examplepackage.module`". This is not how it works. When using `%aimport` you also have to flag it with a `-`. See below. 

In [23]:
%autoreload 2
%aimport examplepackage.neato
%aimport -examplepackage.neato2

In [24]:
examplepackage.neato.torpedo()

First module modification 1


In [25]:
examplepackage.neato2.torpedo2()

Second module modification 0!


In [26]:
examplepackage.neato3.torpedo3()

Third module modification 1!


In [27]:
%%writefile ../examplepackage/neato.py

def torpedo():
    print('First module modification 2!')

Overwriting ../examplepackage/neato.py


In [28]:
%%writefile ../examplepackage/neato2.py

def torpedo2():
    print('Second module modification 2!')

Overwriting ../examplepackage/neato2.py


In [29]:
%%writefile ../examplepackage/neato3.py

def torpedo3():
    print('Third module modification 2!')

Overwriting ../examplepackage/neato3.py


In [30]:
# when hitting 'run all' this needs a short delay (race condition).
time.sleep(1.5)

In [31]:
examplepackage.neato.torpedo()

First module modification 2!


In [32]:
examplepackage.neato2.torpedo2()

Second module modification 0!


In [33]:
examplepackage.neato3.torpedo3()

Third module modification 2!
